In [218]:
import json
import pandas as pd
pd.set_option("display.precision", 3)

In [219]:
games = pd.read_csv(f'../reference_data/NFL/game_logs_2024.csv',index_col=0,parse_dates=['date'])
games['Rush+Rec TDs'] = games['Rec TDs'] + games['Rush TDs']
games['Completion Percentage'] = 100 * (games['Pass Completions'] / games['Pass Attempts'])
games['Pass+Rush Yds'] = games['Pass Yards'] + games['Rush Yards']
games['Rush+Rec Yds'] = games['Receiving Yards'] + games['Rush Yards']
games[['XPM','FG']] = games[['XPM','FG']].astype(float)
games['Kicking Points'] = games['XPM'] + (3 * games['FG'])
games['Fantasy Score'] = ((games['Pass Yards'] * 0.04) + (games['Pass TDs'] * 4) + (2 * -games['INT']) + (games['Rush Yards'] * 0.1) + (games['Receiving Yards'] * 0.1) + #missing 2 pt conv
                    (games['Receptions']) + (-games['Fumbles Lost']) + (games['Rush+Rec TDs'] * 6))

ref2pp = {
    'GNB':'GB',
    'JAX':'JAC',
    'KAN':'KC',
    'LVR':'LV',
    'NOR':'NO',
    'NWE':'NE',
    'SFO':'SF',
    'TAM':'TB',
}
games['team'] = games['team'].apply(lambda x: ref2pp.get(x,x))
games['pct'] = games['pct'].fillna('0%')
games['pct'] = games['pct'].apply(lambda x: float((x.split('%')[0])))
#games = games.loc[games['date'] > '2024-09-29']

In [220]:
gp = games.groupby(['player', 'team', 'pos'])['date'].count()
gp = gp.reset_index().rename({'date':'gp'},axis=1)
gp

,player,team,pos,gp
0,A.J. Brown,PHI,WR,5
1,AJ Barner,SEA,TE,7
2,AJ Cole III,LV,P,9
3,Aaron Jones,MIN,RB,8
4,Aaron Rodgers,NYJ,QB,9
...,...,...,...,...
587,Zach Frazier,PIT,C,1
588,Zack Moss,CIN,RB,8
589,Zamir White,LV,RB,7
590,Zay Flowers,BAL,WR,9


In [221]:
tt = games.groupby([ 'team', 'date'])['Rec Targets'].sum()
tt = tt.reset_index().rename({'Rec Targets':'team_targets'},axis=1)
games = games.merge(tt)
opp_avg = games.groupby(['opp','pos', 'date'])['Fantasy Score'].sum().reset_index().groupby(['opp','pos'])['Fantasy Score'].mean().reset_index()
opp_avg = opp_avg.rename({'Fantasy Score':'pts_allowed'},axis=1)
opp_avg['norm'] = opp_avg.groupby('pos')['pts_allowed'].transform(lambda x: (x / x.mean()))
opp_avg.loc[opp_avg.pos=='WR'].sort_values(by='norm')
games = games.merge(opp_avg, on=['pos','opp'])

In [222]:
ppg = games.groupby(['player', 'team', 'pos'])[['Pass Completions', 'Pass Attempts', 'Pass Yards',
       'Pass TDs', 'INT', 'Sacks', 'Yards Sacked', 'Longest Pass',
       'Passing Rate', 'Rush Attempts', 'Rush Yards', 'Rush TDs',
       'Longest Rush', 'Rec Targets', 'Receptions', 'Receiving Yards',
       'Rec TDs', 'Longest Reception', 'Fumbles', 'Fumbles Lost', 'XPM', 'XPA',
       'FG', 'FGA', 'Kicking Points', 'season','team_targets',
       'snaps', 'pct', 'Rush+Rec TDs', 'Completion Percentage',
       'Pass+Rush Yds', 'Rush+Rec Yds', 'Fantasy Score', 'pts_allowed', 'norm'
]].mean().reset_index().merge(gp)
ppg['ypt'] = ppg['Receiving Yards'] / ppg['Rec Targets']
ppg['target_share'] = ppg['Rec Targets'] / ppg['team_targets']
ppg['norm'] = ppg['Fantasy Score'] * ppg['norm']

In [223]:
qb_cols =     ['player', 'gp','Pass Completions', 'Pass Attempts', 'Pass Yards',
       'Pass TDs', 'INT', 'Sacks', 'Yards Sacked', 'Longest Pass',
       'Passing Rate',  'snaps', 'pct', 'pos']
wr_cols =     ['player','team','gp', 'Rec Targets','target_share', 'Receptions', 'Receiving Yards', 'ypt', 'Rush+Rec Yds', 
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fantasy Score',  'snaps', 'pct', 'pos', 'pts_allowed', 'norm']
rb_cols =     ['player','gp', 'Rush Attempts', 'Rush Yards', 'Rush TDs', 'Rush+Rec Yds', 
       'Longest Rush', 'Rec Targets', 'Receptions', 'Receiving Yards',
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fumbles', 'Fumbles Lost', 'Fantasy Score', 'snaps', 'pct',
        'pos',]

In [224]:
games.loc[games.player=='Cade Otton'][['date','player','team', 'Rec Targets', 'Receptions', 'Receiving Yards', 'Rush+Rec Yds', 
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fantasy Score','team_targets']].sort_values(by='date')

,date,player,team,Rec Targets,Receptions,Receiving Yards,Rush+Rec Yds,Rec TDs,Rush+Rec TDs,Longest Reception,Fantasy Score,team_targets
2807,2024-09-08,Cade Otton,TB,2.0,1.0,5.0,5.0,0.0,0.0,5.0,1.5,30.0
2899,2024-09-15,Cade Otton,TB,2.0,0.0,0.0,-4.0,0.0,0.0,0.0,-0.4,19.0
2205,2024-09-22,Cade Otton,TB,8.0,7.0,47.0,47.0,0.0,0.0,13.0,10.7,32.0
262,2024-09-29,Cade Otton,TB,9.0,6.0,52.0,52.0,0.0,0.0,16.0,11.2,47.0
428,2024-10-03,Cade Otton,TB,4.0,3.0,44.0,44.0,0.0,0.0,22.0,7.4,24.0
876,2024-10-13,Cade Otton,TB,6.0,2.0,15.0,15.0,1.0,1.0,8.0,9.5,36.0
171,2024-10-21,Cade Otton,TB,10.0,8.0,100.0,100.0,0.0,0.0,28.0,18.0,45.0
433,2024-10-27,Cade Otton,TB,10.0,9.0,81.0,81.0,2.0,2.0,29.0,29.1,46.0
68,2024-11-04,Cade Otton,TB,11.0,8.0,77.0,77.0,1.0,1.0,18.0,21.7,31.0


In [225]:
ppg.loc[(ppg.pos=='WR') & (ppg.gp > 3) & (ppg['Fantasy Score'] > 7)].sort_values(by='norm',ascending=False)[wr_cols].head(30).reset_index(drop=True)

,player,team,gp,Rec Targets,target_share,Receptions,Receiving Yards,ypt,Rush+Rec Yds,Rec TDs,Rush+Rec TDs,Longest Reception,Fantasy Score,snaps,pct,pos,pts_allowed,norm
0,Cooper Kupp,LAR,4,12.250,0.325,8.500,75.500,6.163,78.000,0.500,0.500,23.750,19.300,54.500,75.500,WR,38.479,22.945
1,Nico Collins,HOU,5,9.000,0.247,6.400,113.400,12.600,113.400,0.600,0.600,42.000,21.340,48.400,68.600,WR,33.739,22.245
2,Chris Godwin,TB,7,8.857,0.266,7.143,82.286,9.290,82.571,0.714,0.714,28.857,19.686,57.000,87.000,WR,35.260,21.446
3,CeeDee Lamb,DAL,8,10.500,0.265,6.625,82.500,7.857,89.000,0.500,0.500,34.750,18.400,56.875,82.125,WR,35.535,20.201
4,Justin Jefferson,MIN,8,8.625,0.330,6.000,97.875,11.348,98.250,0.625,0.625,39.500,19.685,56.375,90.625,WR,33.046,20.099
5,Malik Nabers,NYG,7,12.000,0.372,7.857,79.571,6.631,79.571,0.429,0.429,23.714,18.386,61.571,93.000,WR,34.373,19.525
6,Ja'Marr Chase,CIN,9,7.778,0.241,6.111,79.667,10.243,80.889,0.778,0.778,32.222,18.867,54.667,89.222,WR,32.756,19.094
7,Amon-Ra St. Brown,DET,8,7.375,0.278,6.000,58.000,7.864,58.000,0.750,0.750,18.125,16.835,55.375,89.250,WR,33.952,17.660
8,A.J. Brown,PHI,5,6.800,0.312,4.600,88.800,13.059,88.800,0.600,0.600,39.400,17.080,53.400,78.600,WR,33.435,17.644
9,Tee Higgins,CIN,5,9.000,0.292,5.800,68.200,7.578,68.200,0.600,0.600,18.800,16.220,47.400,78.600,WR,35.111,17.596


In [226]:
with open('../reference_data/NFL/fantasy.json', 'r') as f:
    data = json.load(f)

with open('../reference_data/NFL/sample.json', 'r') as f:
    teams = json.load(f)
 
data[0]

{'defaultPositionId': 4,
 'droppable': True,
 'eligibleSlots': [5, 6, 23, 7, 20, 21],
 'firstName': 'Curtis',
 'fullName': 'Curtis Hodges',
 'id': 4242896,
 'lastName': 'Hodges',
 'ownership': {'percentOwned': 0.008875975542424907},
 'proTeamId': 0,
 'universeId': 0}

In [227]:
teams['teams'][2]['roster']['entries'][0]['playerPoolEntry']

{'appliedStatTotal': 0.0,
 'id': 3117251,
 'keeperValue': 0,
 'keeperValueFuture': 1,
 'lineupLocked': False,
 'onTeamId': 3,
 'player': {'active': True,
  'defaultPositionId': 2,
  'draftRanksByRankType': {'STANDARD': {'auctionValue': 57,
    'published': False,
    'rank': 1,
    'rankSourceId': 0,
    'rankType': 'STANDARD',
    'slotId': 0},
   'PPR': {'auctionValue': 44,
    'published': False,
    'rank': 12,
    'rankSourceId': 0,
    'rankType': 'PPR',
    'slotId': 0}},
  'droppable': True,
  'eligibleSlots': [2, 3, 23, 7, 20, 21],
  'firstName': 'Christian',
  'fullName': 'Christian McCaffrey',
  'id': 3117251,
  'injured': True,
  'injuryStatus': 'INJURY_RESERVE',
  'lastName': 'McCaffrey',
  'lastNewsDate': 1729552113000,
  'lastVideoDate': 1727792061000,
  'outlooks': {'outlooksByWeek': {'1': "Christian McCaffrey sustained a calf injury this offseason, but the 49ers have not expressed any concern that he won't be available against the Jets in Week 1. The Jets have one of t

In [228]:
cols = [ 'acquisitionType', 
             'playerPoolEntry.player.defaultPositionId',
       'playerPoolEntry.player.draftRanksByRankType.PPR.rank',
        'playerPoolEntry.player.fullName', 'playerPoolEntry.player.injured',
       'playerPoolEntry.player.injuryStatus',
       'playerPoolEntry.player.ownership.averageDraftPosition',
       'playerPoolEntry.player.proTeamId',
       'playerPoolEntry.ratings.0.positionalRanking','name']

In [229]:
len(teams['teams'][13])

IndexError: list index out of range

In [15]:
rosters = pd.DataFrame()
for i in range(12):
    print(i)
    temp = pd.json_normalize(teams['teams'][i], record_path=['roster', 'entries'], meta=['name'])[cols]
    rosters = pd.concat((rosters, temp))
rosters.columns = [ 'acquisitionType',
      'pos_id',
       'PPR.rank',
        'player', 'injured',
       'injuryStatus',
       'ADP',
       'proTeamId',
       'positionalRanking','name']

0
1
2
3
4
5
6
7
8
9
10
11


In [16]:
rosters

,acquisitionType,pos_id,PPR.rank,player,injured,injuryStatus,ADP,proTeamId,positionalRanking,name
0,DRAFT,2,17,Derrick Henry,False,ACTIVE,10.597,33,2,Big Dix Nix
1,DRAFT,3,9,Nico Collins,True,INJURY_RESERVE,25.121,34,10,Big Dix Nix
2,DRAFT,1,29,Lamar Jackson,False,ACTIVE,19.723,33,1,Big Dix Nix
3,DRAFT,3,93,Tee Higgins,False,ACTIVE,100.194,4,24,Big Dix Nix
4,DRAFT,4,248,Kyle Pitts,False,ACTIVE,135.311,1,6,Big Dix Nix
...,...,...,...,...,...,...,...,...,...,...
11,TRADE,3,64,Jaxon Smith-Njigba,False,ACTIVE,105.061,26,27,Team Rambone
12,ADD,5,470,Cameron Dicker,False,ACTIVE,168.185,24,25,Team Rambone
13,ADD,2,398,D'Ernest Johnson,False,ACTIVE,169.978,30,66,Team Rambone
14,ADD,16,253,Bengals D/ST,False,NaN,162.084,4,21,Team Rambone


rosters.loc[rosters.name == 'Hot Chubb Time Machine']

In [17]:
df = rosters.merge(ppg)

In [18]:
df

,acquisitionType,pos_id,PPR.rank,player,injured,injuryStatus,ADP,proTeamId,positionalRanking,name,...,snaps,pct,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Fantasy Score,gp,ypt,target_share
0,DRAFT,2,17,Derrick Henry,False,ACTIVE,10.597,33,2,Big Dix Nix,...,36.500,54.750,1.375,NaN,118.250,126.500,21.900,8,6.600,0.044
1,DRAFT,3,9,Nico Collins,True,INJURY_RESERVE,25.121,34,10,Big Dix Nix,...,48.400,68.600,0.600,NaN,0.000,113.400,21.340,5,12.600,0.247
2,DRAFT,1,29,Lamar Jackson,False,ACTIVE,19.723,33,1,Big Dix Nix,...,66.500,98.750,0.250,69.254,325.000,62.625,25.758,8,NaN,0.000
3,DRAFT,3,93,Tee Higgins,False,ACTIVE,100.194,4,24,Big Dix Nix,...,47.400,78.600,0.600,NaN,0.000,68.200,16.220,5,7.578,0.292
4,DRAFT,4,248,Kyle Pitts,False,ACTIVE,135.311,1,6,Big Dix Nix,...,47.875,72.375,0.375,NaN,0.000,52.375,11.113,8,9.976,0.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,ADD,4,143,Tyler Conklin,False,ACTIVE,149.746,20,17,Team Rambone,...,53.667,85.222,0.222,NaN,0.000,26.333,6.967,9,5.925,0.127
168,TRADE,3,64,Jaxon Smith-Njigba,False,ACTIVE,105.061,26,27,Team Rambone,...,57.500,85.375,0.125,100.000,4.375,48.500,11.150,8,6.258,0.209
169,ADD,5,470,Cameron Dicker,False,ACTIVE,168.185,24,25,Team Rambone,...,0.000,0.000,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN
170,ADD,2,398,D'Ernest Johnson,False,ACTIVE,169.978,30,66,Team Rambone,...,18.429,30.143,0.000,NaN,15.429,26.429,3.786,7,6.417,0.054


In [19]:
recent = games.sort_values(by=['date']).groupby('player').tail(1)
recent = recent[['player','Fantasy Score']].dropna().sort_values(by='Fantasy Score')
recent.columns = ['player','last_game']
recent = ppg.merge(recent)
recent['dud'] = recent['last_game'] - recent['Fantasy Score']
recent

,player,team,pos,Pass Completions,Pass Attempts,Pass Yards,Pass TDs,INT,Sacks,Yards Sacked,...,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Fantasy Score,gp,ypt,target_share,last_game,dud
0,A.J. Brown,PHI,WR,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.750,NaN,0.000,102.000,19.950,4,13.600,0.349,5.60,-14.350
1,AJ Barner,SEA,TE,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.167,NaN,0.000,17.500,4.417,6,10.500,0.044,6.70,2.283
2,Aaron Jones,MIN,RB,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.429,NaN,71.571,104.000,15.971,7,9.080,0.142,12.20,-3.771
3,Aaron Rodgers,NYJ,QB,21.889,35.111,234.111,1.667,0.778,2.222,17.111,...,0.000,62.957,238.889,4.778,14.953,9,NaN,0.000,20.34,5.387
4,Aaron Shampklin,PIT,RB,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,NaN,5.667,5.667,0.567,3,NaN,0.000,-0.20,-0.767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,Zach Frazier,PIT,C,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,NaN,0.000,-2.000,-0.200,1,-inf,0.000,-0.20,0.000
492,Zack Moss,CIN,RB,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.375,NaN,30.250,53.625,10.363,8,6.926,0.107,7.90,-2.463
493,Zamir White,LV,RB,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,NaN,27.333,32.333,3.900,6,3.750,0.041,7.00,3.100
494,Zay Flowers,BAL,WR,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.125,NaN,5.875,71.750,13.050,8,8.639,0.270,29.70,16.650


In [20]:
recent.loc[recent.pos=='RB'].sort_values(by='dud').head(20)

,player,team,pos,Pass Completions,Pass Attempts,Pass Yards,Pass TDs,INT,Sacks,Yards Sacked,...,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Fantasy Score,gp,ypt,target_share,last_game,dud
266,Jordan Mason,SF,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.375,NaN,85.625,96.375,13.013,8,7.167,0.051,1.8,-11.213
315,Kyren Williams,LAR,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,1.429,NaN,76.143,89.000,20.043,7,4.091,0.096,10.5,-9.543
260,Jonathan Taylor,IND,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,1.000,NaN,90.800,108.600,18.260,5,8.091,0.087,8.9,-9.360
419,Tank Bigsby,JAC,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.571,NaN,70.429,75.571,11.414,7,7.200,0.023,2.2,-9.214
246,Jerome Ford,CLE,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.167,NaN,44.000,58.500,10.183,6,3.783,0.119,1.7,-8.483
13,Alexander Mattison,LV,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.500,NaN,33.125,57.500,11.500,8,7.500,0.098,3.6,-7.900
409,Sean Tucker,TB,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.400,NaN,35.000,53.200,8.920,5,13.000,0.034,2.5,-6.420
464,Tyler Goodson,IND,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.200,NaN,25.600,35.000,6.300,5,4.700,0.061,0.0,-6.300
298,Kenneth Walker III,SEA,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,1.167,NaN,52.500,84.667,19.967,6,6.226,0.141,13.7,-6.267
141,Devin Singletary,NYG,RB,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.333,NaN,41.667,56.167,9.450,6,5.800,0.075,3.3,-6.150


In [21]:
df = df.merge(recent)
df

,acquisitionType,pos_id,PPR.rank,player,injured,injuryStatus,ADP,proTeamId,positionalRanking,name,...,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Fantasy Score,gp,ypt,target_share,last_game,dud
0,DRAFT,2,17,Derrick Henry,False,ACTIVE,10.597,33,2,Big Dix Nix,...,1.375,NaN,118.250,126.500,21.900,8,6.600,0.044,26.30,4.400
1,DRAFT,3,9,Nico Collins,True,INJURY_RESERVE,25.121,34,10,Big Dix Nix,...,0.600,NaN,0.000,113.400,21.340,5,12.600,0.247,15.80,-5.540
2,DRAFT,1,29,Lamar Jackson,False,ACTIVE,19.723,33,1,Big Dix Nix,...,0.250,69.254,325.000,62.625,25.758,8,NaN,0.000,23.60,-2.157
3,DRAFT,3,93,Tee Higgins,False,ACTIVE,100.194,4,24,Big Dix Nix,...,0.600,NaN,0.000,68.200,16.220,5,7.578,0.292,18.20,1.980
4,DRAFT,4,248,Kyle Pitts,False,ACTIVE,135.311,1,6,Big Dix Nix,...,0.375,NaN,0.000,52.375,11.113,8,9.976,0.162,2.10,-9.013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,DRAFT,2,89,Devin Singletary,False,ACTIVE,119.423,19,40,Team Rambone,...,0.333,NaN,41.667,56.167,9.450,6,5.800,0.075,3.30,-6.150
153,ADD,4,143,Tyler Conklin,False,ACTIVE,149.746,20,17,Team Rambone,...,0.222,NaN,0.000,26.333,6.967,9,5.925,0.127,0.70,-6.267
154,TRADE,3,64,Jaxon Smith-Njigba,False,ACTIVE,105.061,26,27,Team Rambone,...,0.125,100.000,4.375,48.500,11.150,8,6.258,0.209,37.00,25.850
155,ADD,2,398,D'Ernest Johnson,False,ACTIVE,169.978,30,66,Team Rambone,...,0.000,NaN,15.429,26.429,3.786,7,6.417,0.054,0.60,-3.186


In [53]:
opp_avg = games.groupby(['opp','pos', 'date'])['Fantasy Score'].sum().reset_index().groupby(['opp','pos'])['Fantasy Score'].mean().reset_index()
opp_avg = opp_avg.rename({'Fantasy Score':'pts_allowed'},axis=1)

In [88]:
opp_avg

,opp,pos,pts_allowed,norm
0,ARI,FB,2.600,0.126
1,ARI,K,0.000,NaN
2,ARI,P,0.000,0.029
3,ARI,QB,18.062,0.610
4,ARI,RB,25.812,1.001
...,...,...,...,...
218,WAS,P,0.000,0.029
219,WAS,QB,15.073,-0.377
220,WAS,RB,22.550,0.086
221,WAS,TE,10.243,-0.476
